In [298]:
import os
import csv
import re
import matplotlib.pyplot as plt
import numpy as np

from pybeerxml.parser import Parser

In [299]:
# Create a yeast name-to-id system
from yeast_names import build_yeast_dicts
yeast_name_to_id, brand_to_ids, id_to_yeast_names = build_yeast_dicts()

In [ ]:
ogs = []
fgs = []
yeasts = set()
malts = set()

parser = Parser()
#dir_count = 0
valid_file_count = 0

def clean_replace(s, target):
    return s.replace(target, '').replace("  ", " ").strip()

for dirpath, dirnames, files in os.walk('./data/_matched'):
    #if dir_count < 100: 
    #    dir_count += 1
    #    continue
    #dir_count += 1
    
    for file_name in files:
        file_path = str(os.path.join(dirpath, file_name))
        try:
            recipes = parser.parse(file_path)
        except:
            print("Erroneous file found, removing: " + file_path)
            os.remove(file_path)
            continue
        
        recipe_valid = False
        for recipe in recipes:
            ogs.append(recipe.og)
            fgs.append(recipe.fg)

            for y in recipe.yeasts:

                if not isinstance(y.name, str):
                    # Try turning the name into a code (likely a wyeast strain)...
                    yeast_name = str(int(y.name))
                else:
                    yeast_name = y.name.lower()
                    # Some basic clean-up to start...
                    if "conan" in yeast_name: yeast_name = "vermont ale"
                    elif "super high gravity" in yeast_name or "wlp 099" in yeast_name: yeast_name = "super high gravity ale"
                    elif "super yeast" in yeast_name: yeast_name = "san diego super"
                    elif "chico" in yeast_name: yeast_name = "chico ale"
                    elif "orval" in yeast_name: yeast_name = "brettanomyces bruxellensis"
                    elif "duvel" in yeast_name: yeast_name = "belgian golden ale"
                    elif "dupont" in yeast_name: yeast_name = "french saison ale"
                    else:
                        # Clean up any spelling mistakes, ordering
                        yeast_name = yeast_name.replace("kรถlsch","kolsch").replace("kรถlsh","kolsch").replace("kölsh","kolsch")
                        yeast_name = yeast_name.replace("monastery","monastary").replace("monestary","monastary")
                        yeast_name = yeast_name.replace("california v ale", "california ale v")
                        yeast_name = yeast_name.replace("mã©lange","melange").replace("brettâ€™","brett") \
                            .replace("cã´te","cote").replace("munuch","munich").replace("lellemand","lallemand") \
                            .replace("champagene", "champagne").replace("vemont","vermont")

                if yeast_name not in yeast_name_to_id:
                    # First attempt: Try to find a yeast code to match
                    s = re.search(r"(inis\-|wlp|us\-|([kwst]|oyl|bry)\-|[mg]|\d+/)\s?\-?(\d+(/\d+)?)", yeast_name, flags=re.IGNORECASE)
                    if s != None and len(s.group()) > 0:
                        # We have a potential yeast product id...
                        product_id = s.group().replace("--","-").replace(" ","")
                        if product_id not in yeast_name_to_id:
                            pass
                            #print(f"Product id {product_id} not found for {yeast_name}, file: {file_path}")
                        else:
                            continue

                    # Try removing the word "yeast" (and watch out for any double spaces that may ensue)
                    if clean_replace(yeast_name, "yeast") in yeast_name_to_id:
                        continue
                    # Try replacing "yeast" with "ale" or "lager"
                    if yeast_name.replace("yeast", "ale") in yeast_name_to_id or yeast_name.replace("yeast", "lager") in yeast_name_to_id:
                        continue
                    # Try adding "ale" or "blend" or "lager" to the end
                    if yeast_name+" ale" in yeast_name_to_id or yeast_name+" lager" in yeast_name_to_id or yeast_name+" blend" in yeast_name_to_id:
                        continue

                    # Check for a wyeast code (4 digit code)
                    s = re.search(r"(\d{4})(\D+|$)", yeast_name, flags=re.IGNORECASE)
                    if s != None and len(s.group(1)) > 0:
                        product_id = s.group(1)
                        if product_id in yeast_name_to_id:
                            continue

                    # Try to match the brand name first and then try to figure out
                    # which yeast we're dealing with within that brand 
                    for brand_name, ids in brand_to_ids.items():
                        s = re.search(brand_name, yeast_name, flags=re.IGNORECASE)
                        if s != None and len(s.group()) > 0:
                            # Find the best possible match
                            best_match_count = 0
                            best_match_id = -1
                            for id in ids:
                                count = 0
                                potential_yeast_names = id_to_yeast_names[id]
                                for name in potential_yeast_names:
                                    s = re.search(name, yeast_name, flags=re.IGNORECASE)
                                    if s != None: count += 1
                                if count > best_match_count:
                                    best_match_count = count
                                    best_match_id = id
                            if best_match_id == -1:
                                # Just choose the first id...
                                assert len(ids) > 0
                                best_match_id = ids[0]
                                yeast_name = id_to_yeast_names[best_match_id][0]
                            else:
                                pass
                                #print(f"Found a brand-based match for '{yeast_name}', with brand '{brand_name}', and names: {id_to_yeast_names[best_match_id]}")

                            break


                    # Last attempt - try to match any of the ids directly with the yeast name string
                    found = False
                    best_len = 0
                    best_name = ""
                    for dict_name in yeast_name_to_id:
                        name_opts = f"({dict_name}|{clean_replace(dict_name, 'ale')}|{clean_replace(dict_name,'lager')})"
                        s = re.search(name_opts, yeast_name)
                        if s != None and len(s.group()) > 0:
                            group_len = len(s.group())
                            if best_len < group_len:
                                best_len = group_len
                                best_name = dict_name

                    if best_len == 0:
                        if yeast_name == '- -' or yeast_name == 'default - - -' or '?' in yeast_name or yeast_name == "deleteme":
                            print(f"Deleting file with empty/unknown (e.g., '- -') yeast: {file_path}")
                            import shutil
                            shutil.copy(file_path, "./data/_not_matched")
                        else:
                            print(f"No matches for '{yeast_name}', file: {file_path}, deleting...")
                            os.remove(file_path)
                        break
                    else:
                        recipe_valid = True
                        #print(f"Matched '{yeast_name}' with '{best_name}', file: {file_path}")
        if recipe_valid:
            valid_file_count += 1
            
        #for m in recipe.fermentables:
        #    if m.type == None:
        #        print(f"Malt type none found in file {file_path}")
        #malts.update([m.name for m in recipe.fermentables])
print(f"Valid files found: {valid_file_count}")

In [342]:
from fermentable_names import build_fermentable_name_dicts
fermentable_name_to_id, id_to_fermentable_names = build_fermentable_name_dicts()

In [343]:
dir_count = 0
dir_min = 0
dir_max = 180

for dirpath, dirnames, files in os.walk('./data/_matched'):
    dir_count += 1
    if dir_count < dir_min: 
        continue
    elif dir_count > dir_max:
        break
    
    for file_name in files:
        file_path = str(os.path.join(dirpath, file_name))
        try:
            recipes = parser.parse(file_path)
        except:
            print("Erroneous file found: " + file_path)
            continue
        
        for recipe in recipes:
            for fermentable in recipe.fermentables:
                
                f_name = str(fermentable.name).lower()
                init_name = f_name
                
                # Start with some basic clean-up
                f_name = f_name.replace("carafaâ","carafa").replace("®","").replace("速","").replace("marris","maris") \
                    .replace("roasted oats","malted oats").replace("mdâ„¢","").replace("torrefied","torrified").replace("specl","special") \
                    .replace("carafaz", "carafa").replace("kรถlsch", "kolsch").replace("©","").replace("ch창teau", "chateau").replace("aรงucar","") \
                    .replace("chã¢teau", "chateau")
                f_name = re.sub(r"(\s{2,})", " ", f_name)
                f_name = re.sub(r"[\'\"](w|b)[\'\"]", r"\1", f_name, flags=re.IGNORECASE)
                f_name = re.sub(r"\brosted\b", "roasted", f_name, flags=re.IGNORECASE)
                f_name = re.sub(r"\bcarmel\b", "caramel", f_name, flags=re.IGNORECASE)
                f_name = re.sub(r"\bsirup\b", "syrup", f_name, flags=re.IGNORECASE)
                f_name = re.sub(r"\,?\s*steel\s*cut\s*\(?pinhead\s*oats\)?", "", f_name, flags=re.IGNORECASE)
                f_name = re.sub(r"(md)?™","",f_name,flags=re.IGNORECASE)
                f_name = re.sub(r"(.+)\s*\,\s*(flaked|malted|cooked)\s*", r"\1 \2", f_name, flags=re.IGNORECASE)
                f_name = re.sub(r"\s*\((us|de|uk|nz|be|pl|english|british|german|belgian|patent|american)\)\s*", " ", f_name, flags=re.IGNORECASE)
                f_name = re.sub(r"\s*(english|german)\s*", "", f_name, flags=re.IGNORECASE)
                f_name = re.sub(r"(cara\s+|crystal\s*/\s*caramel|caramel\s*/\s*crystal|caramel|crystal)(\s+malts?\s+\-?)?","caramel/crystal", f_name, flags=re.IGNORECASE)
                f_name = re.sub(r"\s*\(?(bob'?s\s*mills|homemade|malting|cal\s*rose|cargill|((green|black|gold)\s?)?swaen|munton'?s?|warminster|biscoff|patagonia|global malt|thomas\s*fawcett|crisp|best\s*(malz|maltz)|simpsons?|valley\s*(malt)?|briess|bairds|gladfield'?s?|château|chateau|castle|dingemans|brewers\s+malt|rahr|organic|dehusked)\)?\s*\-*\s*", " ", f_name, flags=re.IGNORECASE)
                f_name = re.sub(r"\(?\d+\s+(ebc|lovibond)\)?\:?", "", f_name, flags=re.IGNORECASE)
                
                f_name = f_name.replace(",","").strip()
                
                if f_name in fermentable_name_to_id:
                    continue
                    
                # Try adding "malt" to the end
                if f_name+" malt" in fermentable_name_to_id:
                    continue
                # Try removing "malt" from the end
                if re.sub(r"\s*malt", "", f_name, flags=re.IGNORECASE) in fermentable_name_to_id:
                    continue
                
                best_len_s1 = 0
                best_id_s1  = -1
                best_len_s2 = 0
                best_id_s2  = -1
                for dict_name, dict_id in fermentable_name_to_id.items():
                    name_opts = "(" + dict_name + ")"
                    s1 = re.search(name_opts, f_name)
                    if s1 != None and len(s1.group()) > 0:
                        group_len = len(s1.group())
                        if best_len_s1 < group_len:
                            best_len_s1 = group_len
                            best_id_s1  = dict_id
                            break # TODO: Get rid of this
                    else:
                        s2 = re.search(name_opts, init_name)
                        if s2 != None and len(s2.group()) > 0:
                            group_len = len(s2.group())
                            if best_len_s2 < group_len:
                                best_len_s2 = group_len
                                best_id_s2  = dict_id
                                break # TODO: Get rid of this

                if best_len_s1 == 0 and best_len_s2 == 0:
                    print(f"Name not found: Modified name: '{f_name}', Original name: '{init_name}' in file: {file_path}") 
        

Name not found: Modified name: 'panocha', Original name: 'panocha' in file: ./data/_matched/132/191089.xml
Name not found: Modified name: 'kandy cane', Original name: 'kandy cane' in file: ./data/_matched/132/189688.xml
Name not found: Modified name: 'bavarian humnovy', Original name: 'bavarian humnovy' in file: ./data/_matched/59/115395.xml
Name not found: Modified name: 'rasberry', Original name: 'rasberry' in file: ./data/_matched/104/10664.xml
Name not found: Modified name: 'malted einkorn', Original name: 'malted einkorn' in file: ./data/_matched/145/bt_184468.xml
Name not found: Modified name: 'best', Original name: 'crisp best (us)' in file: ./data/_matched/7/bt_13433.xml
Name not found: Modified name: 'wort', Original name: 'wort' in file: ./data/_matched/7/488969.xml
Name not found: Modified name: 'shepherds dlight', Original name: 'shepherds dlight' in file: ./data/_matched/7/355557.xml
Name not found: Modified name: 'jantar', Original name: 'simpsons jantar' in file: ./data/

In [1]:
from hop_names import build_hop_name_dicts
hop_name_to_id, id_to_hop_names = build_hop_name_dicts()

In [403]:
dir_count = 0
dir_min = 10
dir_max = 20

for dirpath, dirnames, files in os.walk('./data/_matched'):
    dir_count += 1
    if dir_count < dir_min: 
        continue
    elif dir_count > dir_max:
        break
    
    for file_name in files:
        file_path = str(os.path.join(dirpath, file_name))
        try:
            recipes = parser.parse(file_path)
        except:
            print("Erroneous file found: " + file_path)
            continue
        
        for recipe in recipes:
            for hop in recipe.hops:
                h_name = str(hop.name).lower().strip()
                init_name = h_name
                
                h_name = h_name.replace("u.s.","us")
                h_name = re.sub(r"(uk kent(\s*goldings?)?|kent holdings)", "east kent goldings", h_name)
                h_name = re.sub(r"(\(?(\d+\.?\d*%\s*aa|™|â„¢|t\-?90|\s*organic|yakima(\s*valley\s?\-?)?|ychhops|,|bmw|lambic|frech)\)?)", "", h_name)
                h_name = re.sub(r"(\s{2,})", " ", h_name) # Clean up extra spacing
                h_name = re.sub(r"(mittelfr端h|mittelfrã¼h|mittelfruh|mittlefruh)", r"mittlefrüh", h_name, flags=re.IGNORECASE)
                h_name = h_name.strip()
                
                if h_name in hop_name_to_id:
                    continue
                
                best_len_s1 = 0
                best_id_s1  = -1
                best_len_s2 = 0
                best_id_s2  = -1
                for dict_name, dict_id in hop_name_to_id.items():
                    name_opts = "(" + dict_name + ")"
                    s1 = re.search(name_opts, h_name)
                    if s1 != None and len(s1.group()) > 0:
                        group_len = len(s1.group())
                        if best_len_s1 < group_len:
                            best_len_s1 = group_len
                            best_id_s1  = dict_id
                            break # TODO: Get rid of this
                    else:
                        s2 = re.search(name_opts, init_name)
                        if s2 != None and len(s2.group()) > 0:
                            group_len = len(s2.group())
                            if best_len_s2 < group_len:
                                best_len_s2 = group_len
                                best_id_s2  = dict_id
                                break # TODO: Get rid of this

                if best_len_s1 == 0 and best_len_s2 == 0:
                    print(f"Name not found: Modified name: '{h_name}', Original name: '{init_name}' in file: {file_path}") 
                

Name not found: Modified name: 'dried meadowsweet', Original name: 'dried meadowsweet' in file: ./data/_matched/68/212873.xml
Name not found: Modified name: 'dried meadowsweet', Original name: 'dried meadowsweet' in file: ./data/_matched/68/212873.xml
Name not found: Modified name: 'oak chips medium', Original name: 'oak chips medium' in file: ./data/_matched/68/212873.xml
Name not found: Modified name: 'lemongrass', Original name: 'lemongrass' in file: ./data/_matched/68/212873.xml
Name not found: Modified name: 'clementine peel', Original name: 'clementine peel' in file: ./data/_matched/68/473226.xml
Name not found: Modified name: 'young´s', Original name: 'young´s' in file: ./data/_matched/157/533352.xml
Name not found: Modified name: 'wet hops', Original name: 'wet hops' in file: ./data/_matched/157/bt_16970.xml
Name not found: Modified name: 'deleteme', Original name: 'deleteme' in file: ./data/_matched/157/bt_177141.xml
Name not found: Modified name: 'bitter bite', Original name:

In [80]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.hist(ogs, bins=50, range=(1.0, 1.1));
ax2.hist(fgs, bins=20, range=(1.0, 1.03));

In [86]:
ogs = np.array(ogs)
fgs = np.array(fgs)
print(ogs.mean(), ogs.std())
print(fgs.mean(), fgs.std())

1.053665150888797 0.009690755932266558
1.0119204905608377 0.0032176378292414834


In [88]:
n_ogs = (ogs - ogs.mean()) / ogs.std()
n_fgs = (fgs - fgs.mean()) / fgs.std()
print(n_ogs.mean(), n_ogs.std())
print(n_fgs.mean(), n_fgs.std())

-7.0745852793642054e-15 1.0
3.3299653359791936e-14 1.0
